# Классификация изображений
В данном примере мы расскажем как происходит классификация изображений

### Задача
В качестве датасета для классификации изображений выбран [BCCD_Dataset](https://github.com/Shenggan/BCCD_Dataset). BCCD Dataset is under *[MIT licence](./LICENSE)*.

Этот набор данных содержит увеличенные изображения клеток крови. Для каждого из 4 различных типов клеток имеются изображения, сгруппированных в 4 различные папки (в зависимости от типа клетки). Типы клеток - эозинофил, лимфоцит, моноцит и нейтрофил.

[Пример модели машинного обучения](https://www.kaggle.com/brsdincer/classify-blood-cell-subtypes-all-process) по классификации клеток на основе этого датасета. В этом примере классифицируются только WBC (White Blood Cell) на изображении. Мы реализуем классификацию WBC на изображениях с помощью Толоки. [Переработанный датасет](https://www.kaggle.com/paultimothymooney/blood-cells) из этого примера мы и будем использовать.

### Вдохновение
Диагностика заболеваний крови часто включает в себя идентификацию и характеристику образцов крови пациентов.
Автоматизированные методы обнаружения и классификации подтипов клеток крови имеют важное медицинское применение.

### Описание
Для решения данной задачи мы будет использовать один проект. В котором будем показывать изображение клетки и краткую инструкцию для визуального сравнения и просим исполнителей выбрать исходя из описания выбрать какой тип клетки они видят на изображении 1 - Эозинофил, 2 - Лимфоцит, 3 - Моноцит, 4 - Нейтрофил.

<table  align="center">
  <tr><td>
    <img src="./manual/img/eosinophil_ex2.jpg"
         alt="Sample blood cell image"  width="500">
  </td></tr>
  <tr><td align="center">
    <b>Изображение 1.</b> Эозинофил
  </td></tr>
</table>


<table  align="center">
  <tr><td>
    <img src="./manual/manual.png"
         alt="Manual"  width="500">
  </td></tr>
  <tr><td align="center">
    <b>Изображение 2.</b> Инструкцию для визуального сравнения
  </td></tr>
</table>


Особенностью данного проекта является то что задача требует специальных знаний для классификации, но будет выполнятся обычными людьми. Для того чтобы подготовить исполнителя была сделана <a href="./manual/manual.html" target="_blank">подробная инструкция</a> а так же организован принцип Тренировка -> Экзамен -> Выполнение заданий

### Настройка среды

Прежде всего, вам нужно зарегистрироваться в Толоке в качестве заказчика и получить ваш токен. Подробное описание этих действий можно посмотреть в примере [learn the basics.]()

Для хранения изображений можно использовать любой s3. Например в [Yandex.Cloud](https://cloud.yandex.ru/) или [Amazon.AWS](https://aws.amazon.com/s3/)
Вам так же нужно будет создать ключ доступа:
- [yandex](https://cloud.yandex.ru/docs/iam/operations/sa/create-access-key) 
- [amazon](https://docs.aws.amazon.com/IAM/latest/APIReference/API_CreateAccessKey.html)

Чтобы скачать датасет вы должны быть зарегистрованы на Kaggle. Надо зайти в Account. И там нажать Create New API Token - скачается json-файл с вашими настройками.

In [ ]:
!pip install toloka-kit==0.1.5
!pip install crowd-kit==0.0.3
!pip install pandas
!pip install boto3
!pip install ipyplot
!pip install kaggle

In [ ]:
import datetime
import os
import random
import time
import uuid
from zipfile import ZipFile

import boto3
import ipyplot
import pandas

import toloka.client as toloka
import toloka.client.project.template_builder as tb
from crowdkit.aggregation import MajorityVote, DawidSkene

Настроим подключение к s3 и создадим там bucket, если его ещё нет.

In [ ]:
key_id='' # enter your key id
access_key=''  # enter your access key
bucket_name = ''  # enter your bucket name if you have, or leave empty
s3_url = 'https://storage.yandexcloud.net'

session = boto3.session.Session()
s3 = session.client(
    service_name='s3',
    endpoint_url=s3_url,
    aws_access_key_id=key_id,
    aws_secret_access_key=access_key
)

if bucket_name == '':
    bucket_name = f'blade-crowd-test-{uuid.uuid4().hex}'
    response = s3.create_bucket(ACL='public-read', Bucket=bucket_name)
    print(response['Location'])

Настроим подключение к Толоке. И проверим что мы подключились к нужному заказчику.

In [ ]:
token = ''  # enter your toloka token

toloka_client = toloka.TolokaClient(token, 'PRODUCTION')
requester = toloka_client.get_requester()
print(requester)

Скачаем датасет с помощью Kaggle API и распакуем его

In [ ]:
os.environ['KAGGLE_USERNAME'] = ''  # "username" from kaggle.json
os.environ['KAGGLE_KEY'] = ''  # "key" from kaggle.json
!kaggle datasets download -d paultimothymooney/blood-cells
with ZipFile('blood-cells.zip', 'r') as archive:
    archive.extractall('archive')

Подготовим основные настройки: где лежат данные для разметки и данные для годен-сетов, список возможных типов клеток(WBC) и какое количество картинок мы хотим разметить.

In [ ]:
data_dir = './archive/dataset-master/dataset-master/JPEGImages/'
test_dir = './archive/dataset2-master/dataset2-master/images/TEST_SIMPLE/'

typecells = ['EOSINOPHIL', 'LYMPHOCYTE', 'MONOCYTE', 'NEUTROPHIL']
tests_count = 4 # количество примеров контрольных задани 4*typecells = 16
tasks_count = None # количество картинок которое хотим обработать, None - если хотим все

Подготовим текстовые описания для всех полей, которые нам потребуются.

In [ ]:
# toloka text setting
project_name = '🔬Определение типа клеток крови'
project_description = 'Посмотрите на картинку и воспользовавшись инструкцией определите тип клеток крови на ней.'
project_label = 'Какая из клеток крови изображена на картинке?'
project_namecells = ['1 - Эозинофил', '2 - Лимфоцит', '3 - Моноцит', '4 - Нейтрофил']

exam_skill_name = f'{project_name} (Экзамен)'
exam_skill_description = f'Как исполнитель прошел экзамен на проекте {project_name}'
quality_skill_name = f'{project_name} (Качество)'
quality_skill_description = f'Как исполнитель выполнял задания на проекте {project_name}'

train_pool_name = 'Train'
exam_pool_name = 'Exam'
exam_public_description = 'Пройдите экзамен чтобы получить доступ к основным платным заданиям.'

Подготовим подсказки. Их мы используем в тренировке, чтобы подробно объяснить исполнителю как отличить одну клетку от другой.

In [ ]:
hint_dict = {'EOSINOPHIL': '1 – Эозинофил. Ядро, разделенное на две дольки перетяжкой посередине. В розовой толще видны красные и оранжевые точки-включения.', 
            'LYMPHOCYTE': '2 – Лимфоцит. Крупное ядро и отсутствие зернистости. Обладают небольшими размерами.',
            'MONOCYTE': '3 – Моноцит. Ядро не разделенно на фрагменты, крупное, темное, чуть вытянутое, выглядящее в виде боба. Имеют довольно большой размер. ', 
            'NEUTROPHIL': '4 – Нейтрофил. Ядро разделено на несколько (2-4) неодинаковых сегментов, соединенных между собой перетяжками. Внутри розовой толщи хорошо выражена зернистость.'}

### Готовим инструкцию

In [ ]:
public_instruction = open('manual/manual.html').read().strip()

for image in os.listdir('manual/img/'):
    s3.upload_file(f'manual/img/{image}', bucket_name, f'manual/img/{image}')

public_instruction = public_instruction.replace('./img/', f'{s3_url}/{bucket_name}/manual/img/')

s3.upload_file('manual/manual.png', bucket_name, 'manual/manual.png')
manualpng = f'{s3_url}/{bucket_name}/manual/manual.png'

### Создаем проект
В этом проекте исполнители выбирают тип клеток крови.

На этом этапе нам нужно настроить, как исполнители будут видеть задачу, написать инструкции и определить формат ввода и вывода. Важно, чтобы мы написали четкие инструкции с примерами, чтобы убедиться, что исполнители делают именно то, что мы хотим.

In [ ]:
# How performers will see the task
radio_group_field = tb.fields.RadioGroupFieldV1(
    data=tb.data.OutputData(path='result'),
    label=project_label,
    validation=tb.conditions.RequiredConditionV1(),
    options=[
        tb.fields.GroupFieldOption(label=cell_name, value=cell_type)
        for cell_name, cell_type in zip(project_namecells, typecells)        
    ]
)

project_interface = toloka.project.view_spec.TemplateBuilderViewSpec(
    config=tb.TemplateBuilder(
        view=tb.view.ListViewV1(
            items=[
                tb.view.ImageViewV1(url=tb.data.InputData(path='image'), max_width=500),
                tb.view.ImageViewV1(url=manualpng, max_width=500),
                radio_group_field,
            ]
        ),
        plugins=[
            tb.plugins.HotkeysPluginV1(
                **{
                    f'key_{i+1}': tb.actions.SetActionV1(data=tb.data.OutputData(path='result'),payload=cell_type)
                    for i, cell_type in enumerate(typecells)
                }
            ),
            tb.plugins.TolokaPluginV1(
                layout = tb.plugins.TolokaPluginV1.TolokaPluginLayout(
                    kind='scroll', 
                    task_width=500,
                )
            ),
        ]
    )
)

# Set up the project
markup_project = toloka.project.Project(
    assignments_issuing_type=toloka.project.Project.AssignmentsIssuingType.AUTOMATED,
    public_name=project_name,
    public_description=project_description,
    public_instructions=public_instruction,
    # Set up the task: view, input, and output parameters
    task_spec=toloka.project.task_spec.TaskSpec(
        input_spec={
            'image': toloka.project.field_spec.StringSpec()
        },
        output_spec={
            'result': toloka.project.field_spec.StringSpec(allowed_values=typecells)
        },
        view_spec=project_interface,
    ),
)

# Call the API to create a new project
markup_project = toloka_client.create_project(markup_project)
print(f'Created markup project with id {markup_project.id}')
print(f'To view the project, go to: https://toloka.yandex.com/requester/project/{markup_project.id}')

### Создаем навыки
Навыки определяются числом от 0 до 100. Например, в качестве навыка можно записать процент правильных ответов. Узнайте больше в справке.

В наших проектах мы будем использовать два навыка:
Тренировочный навык. Показывает, насколько хорошо прошел исполнитель тренировку.

Экзаменационный навык. Показывает, насколько хорошо прошел исполнитель экзамен.

In [ ]:
exam_skill = next(toloka_client.get_skills(name=exam_skill_name), None)
if exam_skill:
    print('Exam skill already exists')
else:
    print('Create new exam skill')
    exam_skill = toloka_client.create_skill(
        name=exam_skill_name,
        hidden=True,
        private_comment=exam_skill_description,
    )

quality_skill = next(toloka_client.get_skills(name=quality_skill_name), None)
if quality_skill:
    print('Quality skill already exists')
else:
    print('Create new quality skill')
    quality_skill = toloka_client.create_skill(
        name=quality_skill_name,
        hidden=True,
        private_comment=quality_skill_description,
    )

### Создаем Тренировку
В нем исполнители отвечают на задания и видят подсказку если ответили не верно. Таким образом мы обучаем исполнителей выполнять наше задание.

In [ ]:
train_pool = toloka.training.Training(
    project_id=markup_project.id,
    private_name=train_pool_name,
    may_contain_adult_content=False,
    assignment_max_duration_seconds=60*20,
    mix_tasks_in_creation_order=False,
    shuffle_tasks_in_task_suite=True,
    training_tasks_in_task_suite_count=15,
    task_suites_required_to_pass=7,
    retry_training_after_days=1,
)

train_pool = toloka_client.create_training(train_pool)
print(f'Created "{train_pool.private_name}" training with id {train_pool.id}')

Добавляем задания в тренировку. Для тренировочных заданий кроме всего прочего нужны подсказки.

In [ ]:
training_tasks = []

for cell_type in typecells:
    dir_path = f'{test_dir}/{cell_type}/'
    test_images_list = os.listdir(dir_path)
    random.shuffle(test_images_list)
    count = tests_count if len(test_images_list) > tests_count else len(test_images_list)
    for image in test_images_list[:count]:
        s3.upload_file(f'{dir_path}{image}', bucket_name, f'train/{image}')
        training_tasks.append(
            toloka.task.Task(
                input_values={'image': f'{s3_url}/{bucket_name}/train/{image}'},
                known_solutions = [toloka.task.BaseTask.KnownSolution(output_values={'result': cell_type})],
                message_on_unknown_solution = hint_dict[cell_type],
                pool_id=train_pool.id,
                infinite_overlap=True,
            )
        )
created_training_tasks = toloka_client.create_tasks(training_tasks, toloka.task.CreateTasksParameters(allow_defaults=True))
print(f'{len(created_training_tasks.items)} tasks added to the pool {train_pool.id}')

### Создаем пул Экзамен
В нем исполнители отвечают на задания и получают или не получают возможность выполнять реальные задания в зависимости от того как они ответили на контрольные задания. К экзамену допускаются исполнители которые ответили правильно хотябы на 50% в тренировке.

In [ ]:
from toloka.client.collectors import AssignmentSubmitTime, GoldenSet
from toloka.client.actions import RestrictionV2, SetSkillFromOutputField
from toloka.client.conditions import (
    FastSubmittedCount,
    GoldenSetCorrectAnswersRate,
    RuleConditionKey,
    TotalAnswersCount,
)

exam_pool = toloka.pool.Pool(
    project_id=markup_project.id,
    private_name=exam_pool_name,
    public_description=exam_public_description,
    may_contain_adult_content=False,
    type='EXAM',
    will_expire=datetime.datetime.utcnow() + datetime.timedelta(days=365),
    reward_per_assignment=0.00,
    auto_accept_solutions=True,
    assignment_max_duration_seconds=60*10,
    defaults=toloka.pool.Pool.Defaults(
         default_overlap_for_new_task_suites=99,
         default_overlap_for_new_tasks=None,
    ),
)

# Set the number of tasks per page
exam_pool.set_mixer_config(real_tasks_count=0, 
                            golden_tasks_count=15,
                            training_tasks_count=0,
                            min_golden_tasks_count=15,
                            mix_tasks_in_creation_order=False,
                            shuffle_tasks_in_task_suite=True,
                            )

exam_pool.filter = (
    toloka.filter.FilterOr([toloka.filter.Languages.in_('RU')]) &
    #toloka.filter.FilterOr([toloka.filter.Languages.in_('EN')]) &
    toloka.filter.FilterOr([
        toloka.filter.ClientType == 'BROWSER',
        toloka.filter.ClientType == 'TOLOKA_APP'
    ])
)

exam_pool.set_training_requirement(
    training_pool_id=train_pool.id,
    training_passing_skill_value=50
)

exam_pool.quality_control.add_action(
    collector=AssignmentSubmitTime(fast_submit_threshold_seconds=7),
    conditions=[FastSubmittedCount > 0],
    action=RestrictionV2(
        scope='PROJECT',
        duration_unit='PERMANENT',
        private_comment='Fast responses'
    )
)

exam_pool.quality_control.add_action(
    collector=GoldenSet(history_size=15),
    conditions=[TotalAnswersCount >= 14],
    action=SetSkillFromOutputField(
        skill_id=exam_skill.id,
        from_field=RuleConditionKey('correct_answers_rate')
    )
)

exam_pool = toloka_client.create_pool(exam_pool)
print(f'Created "{exam_pool.private_name}" pool with id {exam_pool.id}')

Добавляем задания в экзамен

In [ ]:
exam_tasks = []

for cell_type in typecells:
    dir_path = f'{test_dir}/{cell_type}/'
    test_images_list = os.listdir(dir_path)
    random.shuffle(test_images_list)
    count = tests_count if len(test_images_list) > tests_count else len(test_images_list)
    for image in test_images_list[:count]:
        s3.upload_file(f'{dir_path}{image}', bucket_name, f'exam/{image}')
        exam_tasks.append(
            toloka.task.Task(
                input_values={'image': f'{s3_url}/{bucket_name}/exam/{image}'},
                known_solutions = [
                    toloka.task.BaseTask.KnownSolution(output_values={'result': cell_type})
                ],
                pool_id=exam_pool.id,
                infinite_overlap=True,
            )
        )

created_exam_tasks = toloka_client.create_tasks(exam_tasks)
print(f'{len(created_exam_tasks.items)} tasks added to the pool {exam_pool.id}')

### Создаем основной пул с заданиями
В нем доверенные исполнители прошедшие тренировку и экзамен размечают настоящие данные. 

In [ ]:
blood_pool = toloka.pool.Pool(
    project_id=markup_project.id,
    private_name=project_name+' - тест добавления заданий',
    may_contain_adult_content=False,
    will_expire=datetime.datetime.utcnow() + datetime.timedelta(days=365),
    reward_per_assignment=0.01,
    auto_accept_solutions=True,
    assignment_max_duration_seconds=60*20,
    defaults=toloka.pool.Pool.Defaults(
         default_overlap_for_new_task_suites=1,
         default_overlap_for_new_tasks=10,
    ),
)

# Set the number of tasks per page
blood_pool.set_mixer_config(real_tasks_count=8, 
                            golden_tasks_count=2,
                            training_tasks_count=0,
                            min_real_tasks_count=1,
                            min_golden_tasks_count=2)

blood_pool.filter = (
    toloka.filter.FilterOr([toloka.filter.Languages.in_('RU')]) &
    #toloka.filter.FilterOr([toloka.filter.Languages.in_('EN')]) &
    toloka.filter.FilterOr([toloka.filter.Skill(exam_skill.id) >= 80]) &
    toloka.filter.FilterOr([
        toloka.filter.ClientType == 'BROWSER',
        toloka.filter.ClientType == 'TOLOKA_APP'
    ]) &
    toloka.filter.FilterOr([
        toloka.filter.Skill(quality_skill.id) >= 80,
        toloka.filter.Skill(quality_skill.id) == None
    ])
)

blood_pool.quality_control.add_action(
    collector=AssignmentSubmitTime(fast_submit_threshold_seconds=10),
    conditions=[FastSubmittedCount > 0],
    action=RestrictionV2(
        scope='PROJECT',
        duration_unit='PERMANENT',
        private_comment='Fast responses'
    )
)

blood_pool.quality_control.add_action(
    collector=GoldenSet(history_size=100),
    conditions=[GoldenSetCorrectAnswersRate < 80],
    action=RestrictionV2(
        scope='PROJECT',
        duration_unit='PERMANENT',
        private_comment='Wrong honeypot'
    )
)

blood_pool.quality_control.add_action(
    collector=GoldenSet(history_size=100),
    conditions=[TotalAnswersCount >= 3],
    action=SetSkillFromOutputField(
        skill_id=quality_skill.id,
        from_field=RuleConditionKey('correct_answers_rate')
    )
)

blood_pool = toloka_client.create_pool(blood_pool)
print(f'Created "{blood_pool.private_name}" pool with id {blood_pool.id}')

Добавляем задания в пул, а так же добавляем контрольные задания. В этом датасете есть пограничные случаи, когда на изображении показано несколько клеток или вообще нет клеток. При выполнении реальной разметки, такие случаи надо отдельно обрабатывать в отдельных проектах или в этом же. Мы для простоты примера выкинем их из входных данных. Сразу же подготовим DataFrame с которым будем сравнивать итоговую разметку.

In [ ]:
# загружаем grounf truth разметку
ground_truth_df = pandas.read_csv('archive/dataset-master/dataset-master/labels.csv', sep=',')
ground_truth_df = ground_truth_df[['Image', 'Category']]
ground_truth_df = ground_truth_df.rename(columns = {'Image':'task','Category':'ground_truth'})

prefix = f'{s3_url}/{bucket_name}/task/BloodImage_'
ground_truth_df['task'] = ground_truth_df['task'].apply(lambda x: f'{prefix}{str(x).zfill(5)}.jpg')

ground_truth_df.set_index('task', inplace=True)
print(ground_truth_df)

In [ ]:
real_tasks = []

# добавляем голден-таски
for cell_type in typecells:
    dir_path = f'{test_dir}/{cell_type}/'
    test_images_list = os.listdir(dir_path)
    for image in test_images_list:
        s3.upload_file(f'{dir_path}{image}', bucket_name, f'golden_task/{image}')
        real_tasks.append(
            toloka.task.Task(
                input_values={'image': f'{s3_url}/{bucket_name}/golden_task/{image}'},
                known_solutions = [toloka.task.BaseTask.KnownSolution(output_values={'result': cell_type})],
                pool_id=blood_pool.id,
                infinite_overlap=True,
            )
        )

# Добавляем основные задания, разметку для которых мы хотим получить
images_list = os.listdir(f'{data_dir}')
count = len(images_list) if tasks_count is None or tasks_count > len(images_list) else tasks_count
for image in images_list[:count]:
    image_url = f'{s3_url}/{bucket_name}/task/{image}'
    if image_url not in ground_truth_df.index or ground_truth_df.loc[image_url, :]['ground_truth'] not in typecells:
        break

    s3.upload_file(f'{data_dir}{image}', bucket_name, f'task/{image}')
    real_tasks.append(
        toloka.task.Task(
            input_values={'image': image_url},
            pool_id=blood_pool.id,
        )
    )
created_tasks = toloka_client.create_tasks(real_tasks, toloka.task.CreateTasksParameters(allow_defaults=True))
print(f'{len(created_tasks.items)} tasks added to the pool {blood_pool.id}')

### Запускаем разметку
Открываем тренировочный, экзаменационный и боевой пул. Ждем выполнения боевого и закрываем все пулы.

In [ ]:
def wait_pool_for_close(pool, timeout_minutes=5):
    sleep_time = 60*timeout_minutes
    pool = toloka_client.get_pool(pool.id)
    while not pool.is_closed():
        print(
            f'{datetime.datetime.now().strftime("%H:%M:%S")} '
            f'Pool {pool.id} has status {pool.status}.'
        )
        time.sleep(sleep_time)
        pool = toloka_client.get_pool(pool.id)

toloka_client.open_pool(train_pool.id)
toloka_client.open_pool(exam_pool.id)
toloka_client.open_pool(blood_pool.id)


# Wait for the pool
print('\nWaiting for the main pool to close')
wait_pool_for_close(blood_pool)
print(f'Pool "{blood_pool.private_name}" is finally closed!')

toloka_client.close_pool(train_pool.id)
print(f'Pool "{train_pool.private_name}" is closed!')

toloka_client.close_pool(exam_pool.id)
print(f'Pool "{exam_pool.private_name}" is closed!')

### Получаем результаты

In [ ]:
answers_df = toloka_client.get_assignments_df(blood_pool.id)

answers_df = answers_df[answers_df['GOLDEN:result'].isnull()].copy()
answers_df = answers_df[['INPUT:image','OUTPUT:result','ASSIGNMENT:worker_id']]
answers_df = answers_df.rename(columns = {'INPUT:image':'task','OUTPUT:result':'label','ASSIGNMENT:worker_id':'performer'})

# Dawid Skene aggregation
ds_labels = DawidSkene(n_iter=20).fit_predict(answers_df)
result = pandas.concat([result, ds_labels], axis=1).rename(columns = {0:'ds_label'})

result = result.drop(result[result.ds_label.isnull()].index)
print(result)